# 1. Install and Import Dependencies

In [228]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler, LabelEncoder
le = LabelEncoder()
import pickle

# std = StandardScaler()

# 2. Load the data

In [229]:
df = pd.read_csv("datasets/Crop_recommendation.csv") 

# 3. Pre-processing the data

In [230]:
x = df.drop(columns="label")
y = pd.DataFrame(df['label'],columns=['label'])
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0, train_size = .75)
print(y_train)
y_train["label"] = le.fit_transform(y_train["label"])
# 
y_test["label"] = le.transform(y_test["label"])
print(le.classes_)
#np.save('classes.npy', le.classes_)

# y_test

          label
1975     cotton
1287     grapes
760   blackgram
267    chickpea
2007       jute
...         ...
1033     banana
1731     papaya
763   blackgram
835      lentil
1653     orange

[1650 rows x 1 columns]
['apple' 'banana' 'blackgram' 'chickpea' 'coconut' 'coffee' 'cotton'
 'grapes' 'jute' 'kidneybeans' 'lentil' 'maize' 'mango' 'mothbeans'
 'mungbean' 'muskmelon' 'orange' 'papaya' 'pigeonpeas' 'pomegranate'
 'rice' 'watermelon']


In [231]:
x_train

,N,P,K,temperature,humidity,ph,rainfall
1975,134,56,18,23.808346,83.919026,6.691268,70.973583
1287,29,122,196,41.948657,81.155952,5.638328,73.068630
760,25,68,19,29.399827,64.255107,7.108450,67.476773
267,41,69,82,20.023815,16.632945,6.715587,68.978065
2007,70,43,40,24.355641,88.803910,6.176860,169.116803
...,...,...,...,...,...,...,...
1033,102,71,48,28.654563,79.286937,5.695268,102.463378
1731,34,68,51,27.347349,94.177567,6.687088,40.351531
763,35,64,15,28.474423,63.536045,6.500145,69.527441
835,39,65,23,25.434598,69.126134,7.685959,41.026829


In [232]:
y_train

,label
1975,6
1287,7
760,2
267,3
2007,8
...,...
1033,1
1731,17
763,2
835,10


In [233]:
y_train = to_categorical(y_train, 22)
y_test = to_categorical(y_test, 22)
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# 4. Model

In [234]:
def crop_predection__model():

    # Define the model
    model = tf.keras.models.Sequential([ 
      tf.keras.layers.Dense(32 , activation='relu'),
      tf.keras.layers.Dense(64 , activation='relu'),
      tf.keras.layers.Dense(22 , activation='softmax'),
  
    ]) 


    # Compile the model
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy']) 
        
    return model

In [235]:
model =crop_predection__model()

In [236]:
history = model.fit(x_train, y_train, epochs=8)

Epoch 1/8
52/52 [==============================] - 1s 3ms/step - loss: 13.9621 - accuracy: 0.0982
Epoch 2/8
52/52 [==============================] - 0s 3ms/step - loss: 1.7720 - accuracy: 0.5024
Epoch 3/8
52/52 [==============================] - 0s 3ms/step - loss: 0.8784 - accuracy: 0.7091
Epoch 4/8
52/52 [==============================] - 0s 3ms/step - loss: 0.5633 - accuracy: 0.8121
Epoch 5/8
52/52 [==============================] - 0s 3ms/step - loss: 0.4005 - accuracy: 0.8764
Epoch 6/8
52/52 [==============================] - 0s 3ms/step - loss: 0.3248 - accuracy: 0.9079
Epoch 7/8
52/52 [==============================] - 0s 3ms/step - loss: 0.2857 - accuracy: 0.9170
Epoch 8/8
52/52 [==============================] - 0s 3ms/step - loss: 0.2589 - accuracy: 0.9145


# 5. Testing

In [237]:
model.evaluate(x_test, y_test, batch_size=50)  

11/11 [==============================] - 0s 2ms/step - loss: 0.2516 - accuracy: 0.9164


[0.25162285566329956, 0.9163636565208435]

In [241]:
print(x_train.iloc[[0]])
pred = model.predict(x_train.iloc[[0]])
pred
# x_train.iloc[[0]]

        N   P   K  temperature   humidity        ph   rainfall
1975  134  56  18    23.808346  83.919026  6.691268  70.973583
1/1 [==============================] - 0s 30ms/step


array([[1.7942151e-11, 4.4482582e-05, 6.9322061e-07, 1.3357239e-09,
        1.1497234e-09, 1.6515105e-04, 9.8119295e-01, 2.4364597e-12,
        3.9452972e-14, 4.6081352e-09, 1.2870565e-06, 1.8492373e-02,
        6.8979466e-07, 8.5543128e-09, 1.0711610e-06, 4.0037797e-08,
        4.9067362e-06, 1.1858741e-06, 8.4517937e-10, 3.2870733e-16,
        7.2618959e-11, 9.5138566e-05]], dtype=float32)

In [239]:
max3_values = np.argsort(pred[0])[::-1]
max3_values

array([ 6, 11,  5, 21,  1, 16, 10, 17, 14,  2, 12, 15, 13,  9,  3,  4, 18,
       20,  0,  7,  8, 19], dtype=int64)

In [240]:
le.inverse_transform(max3_values)

array(['cotton', 'maize', 'coffee', 'watermelon', 'banana', 'orange',
       'lentil', 'papaya', 'mungbean', 'blackgram', 'mango', 'muskmelon',
       'mothbeans', 'kidneybeans', 'chickpea', 'coconut', 'pigeonpeas',
       'rice', 'apple', 'grapes', 'jute', 'pomegranate'], dtype=object)

# Pickle file

In [225]:
# pickle.dump(model, open("model.pkl", "wb"))

In [226]:
# model.save("saved_model/crop_predection__model")

In [227]:
# pickle.dump(model , open("model.pkl" , "wb"))
model.save("my_h5_model.h5")
output = open('Departure_encoder.pkl', 'wb')
pickle.dump(le, output)
output.close()